# Segmenting and Clustering Part 3

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!pip install folium
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Folium installed
Libraries imported.


## Step 1 Create a map of Toronto (with marker)

In [38]:
df5 = pd.read_csv('geodata.csv')
df5.head()

,Postalcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.752440,-79.329271
1,M4A,North York,Victoria Village,43.730421,-79.313320
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.655120,-79.362640
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.723125,-79.451589
4,M7A,Queen's Park,Queen's Park,43.661102,-79.391035


In [39]:
address = 'Toronto, Ontario'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


In [42]:
map_toronto = folium.Map(location=[location.latitude, location.longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighbourhood in zip(df5['Latitude'], df5['Longitude'], df5['Borough'], df5['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Step 2 FourSquare API to explore Toronto Neibourhood

In [78]:
# The code was removed by Watson Studio for sharing.

Your credentails:
CLIENT_ID: EOQZ5QGTB5IB0CL5JGVLEFUY5T4QEPQODIQC0GUYKRAZFJCF
CLIENT_SECRET:ZAI1EJPX5QEFPSUGR3DF2GZ404HUNETZQ3KGBLYRXEXFQHAQ


In [17]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [18]:
df5 = pd.read_csv('geodata.csv')
df5.head()

,Postalcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.752440,-79.329271
1,M4A,North York,Victoria Village,43.730421,-79.313320
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.655120,-79.362640
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.723125,-79.451589
4,M7A,Queen's Park,Queen's Park,43.661102,-79.391035


In [20]:
venues_list = getNearbyVenues(names=df5['Neighbourhood'],
                                   latitudes=df5['Latitude'],
                                   longitudes=df5['Longitude']
                                  )

Parkwoods
Victoria Village
Harbourfront, Regent Park
Lawrence Heights, Lawrence Manor
Queen's Park
Islington Avenue
Rouge, Malvern
Don Mills North
Woodbine Gardens, Parkview Hill
Ryerson, Garden District
Glencairn
Cloverdale, Islington, Martin Grove, Princess Gardens, West Deane Park
Highland Creek, Rouge Hill, Port Union
Flemingdon Park, Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Downsview North, Wilson Heights
Thorncliffe Park
Adelaide, King, Richmond
Dovercourt Village, Dufferin
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto
Harbourfront East, Toronto Islands, Union Station
Little Portugal, Trinity
East Birchmount Park, Ionview, Kennedy Park
Bayview Village
CFB Toronto, Downsview East
The D

## Step 3 Closer Analysis 

In [21]:
venues_list.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.752440,-79.329271,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.752440,-79.329271,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Parkwoods,43.752440,-79.329271,KFC,43.754387,-79.333021,Fast Food Restaurant
3,Victoria Village,43.730421,-79.313320,Wigmore Park,43.731023,-79.310771,Park
4,Victoria Village,43.730421,-79.313320,Memories of Africa,43.726602,-79.312427,Grocery Store


In [23]:
venues_list.shape

(2477, 7)

In [26]:
venues_list['Venue Category'].unique()

array(['Park', 'Food & Drink Shop', 'Fast Food Restaurant',
       'Grocery Store', 'Intersection', 'French Restaurant', 'Bakery',
       'Coffee Shop', 'Breakfast Spot', 'Spa', 'Mexican Restaurant',
       'Yoga Studio', 'Event Space', 'Pub', 'Italian Restaurant',
       'Gym / Fitness Center', 'Thai Restaurant', 'Restaurant', 'Theater',
       'Food Truck', 'Electronics Store', 'Thrift / Vintage Store',
       'Health Food Store', 'Clothing Store', 'Shopping Mall',
       'Toy / Game Store', 'Cosmetics Shop', 'American Restaurant',
       'Jewelry Store', "Men's Store", 'Tea Room', 'Kitchen Supply Store',
       'Furniture / Home Store', 'Pharmacy', 'Leather Goods Store',
       'Greek Restaurant', 'Mediterranean Restaurant', 'Chocolate Shop',
       'Dessert Shop', 'Food Court', 'Bookstore', 'Department Store',
       'Movie Theater', "Women's Store", 'Burger Joint',
       'Frozen Yogurt Shop', 'Sushi Restaurant', 'Café',
       'Fried Chicken Joint', 'Lingerie Store', 'Platform',


In [28]:
print('In total {} uniques categories.'.format(len(venues_list['Venue Category'].unique())))

In total 255 uniques categories.


In [31]:
# one hot encoding
toronto_onehot = pd.get_dummies(venues_list[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = venues_list['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,Transportation Service,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
toronto_neiggrouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_neiggrouped

,Neighbourhood,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,Transportation Service,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.000000,0.000000,0.030000,0.000000,0.010000,0.000000,0.000000,0.030000,0.000000,...,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.0,0.000000
1,Agincourt,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.071429,0.000000,0.000000,0.0,0.000000
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.071429,0.000000,0.000000,0.000000,0.0,0.000000
4,"Alderwood, Long Branch",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
5,Bayview Village,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
6,"Bedford Park, Lawrence Manor East",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
7,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.015873,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.015873,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
8,"Birch Cliff, Cliffside West",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
9,"Bloordale Gardens, Eringate, Markland Wood, Ol...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000


In [63]:
for neigh in toronto_neiggrouped['Neighbourhood']:
    print("----"+neigh+"----")
    tbl = toronto_neiggrouped[toronto_neiggrouped['Neighbourhood'] == neigh].T.reset_index()
    tbl.columns = ['venue','freq']
    tbl = tbl.iloc[1:]
    tbl['freq'] = tbl['freq'].astype(float)
    tbl = tbl.round({'freq': 2})
    print(tbl.sort_values('freq', ascending=False).reset_index(drop=True).head())
    print('\n')

----Adelaide, King, Richmond----
         venue  freq
0  Coffee Shop  0.07
1         Café  0.06
2   Steakhouse  0.04
3        Hotel  0.04
4   Restaurant  0.03


----Agincourt----
                 venue  freq
0        Shopping Mall  0.14
1   Chinese Restaurant  0.14
2          Supermarket  0.14
3         Skating Rink  0.07
4  Shanghai Restaurant  0.07


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                       venue  freq
0                   Pharmacy   1.0
1          Afghan Restaurant   0.0
2                  Nightclub   0.0
3         Mexican Restaurant   0.0
4  Middle Eastern Restaurant   0.0


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
                  venue  freq
0         Grocery Store  0.14
1           Coffee Shop  0.07
2  Fast Food Restaurant  0.07
3          Liquor Store  0.07
4           Pizza Place  0.07


----Alderwood, Long Branch----
               venue  freq
0  Conveni

# Step 4 Clustering

In [77]:
kclusters = 6

torontoneig_clustering = toronto_neiggrouped.drop('Neighbourhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=1).fit(torontoneig_clustering)

# check cluster labels generated for each row in the dataframe
print(kmeans.labels_)

[3 3 0 3 3 3 3 3 3 3 3 3 3 3 3 3 3 5 3 3 3 3 3 0 3 0 3 3 3 3 3 3 3 0 3 3 3
 3 3 3 3 3 3 3 3 1 3 3 3 0 3 1 2 3 3 3 3 3 0 0 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 4 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 0 3 3 3 0 3 3]


In [76]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(df5['Latitude'], df5['Longitude'], df5['Neighbourhood'],kmeans.labels_):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters